In [ ]:
import pandas as pd, pickle, numpy as np, warnings, time, os
warnings.filterwarnings("ignore", "FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas")
pd.options.mode.copy_on_write = True
from config_path import PATH_CLEAN, PATH_API
from functions_shared import chunkify, work_csv
from step7_persons.prep_persons import persons_preparation
from step7_persons.affiliations import affiliations, persons_files_import, persons_api_simplify, persons_results_clean

PATH_PERSONS=f"{PATH_API}persons/"
CSV_DATE='20250121'
def persons_preparation(csv_date):

    import pandas as pd, numpy as np, warnings
    warnings.filterwarnings("ignore", "This pattern is interpreted as a regular expression, and has match groups")
    pd.options.mode.copy_on_write = True
    from constant_vars import FRAMEWORK
    from config_path import PATH_SOURCE, PATH_CLEAN
    from functions_shared import unzip_zip, my_country_code, country_iso_shift, prop_string

    ###############################
    participation = pd.read_pickle(f"{PATH_CLEAN}participation_current.pkl")
    entities = pd.read_pickle(f"{PATH_CLEAN}entities_info_current2.pkl")
    project = pd.read_pickle(f"{PATH_CLEAN}projects_current.pkl")
    my_countries=my_country_code()

    print(f"size participation: {len(participation)}")
    ######################
    print(f"\n### IMPORT datasets")
    perso_part = unzip_zip(f'he_grants_ecorda_pd_{csv_date}.zip', f"{PATH_SOURCE}{FRAMEWORK}/", "participant_persons.csv", 'utf-8')
    perso_part = (perso_part.loc[perso_part.FRAMEWORK=='HORIZON',
            ['PROJECT_NBR', 'GENERAL_PIC', 'PARTICIPANT_PIC', 'ROLE', 'FIRST_NAME',
            'LAST_NAME', 'TITLE', 'GENDER', 'PHONE', 'EMAIL',
            'BIRTH_COUNTRY_CODE', 'NATIONALITY_COUNTRY_CODE', 'HOST_COUNTRY_CODE', 'SENDING_COUNTRY_CODE']]
                .rename(columns=str.lower)
                .rename(columns={'project_nbr':'project_id', 'general_pic':'generalPic', 'participant_pic':'pic'})
                .assign(stage='successful'))
    print(f"size perso_part import: {len(perso_part)}")

    ######################################
    perso_app = unzip_zip(f'he_proposals_ecorda_pd_{csv_date}.zip', f"{PATH_SOURCE}{FRAMEWORK}/", "applicant_persons.csv", 'utf-8')

    perso_app = (perso_app.loc[perso_app.FRAMEWORK=='HORIZON',
        ['PROPOSAL_NBR', 'GENERAL_PIC', 'APPLICANT_PIC', 'ROLE', 'FIRST_NAME',
        'FAMILY_NAME', 'TITLE', 'GENDER', 'PHONE', 'EMAIL',
        'RESEARCHER_ID', 'ORCID_ID', 'GOOGLE_SCHOLAR_ID','SCOPUS_AUTHOR_ID']]
                .rename(columns=str.lower)
                .rename(columns={'proposal_nbr':'project_id', 'general_pic':'generalPic', 'applicant_pic':'pic', 'family_name':'last_name'})
                .assign(stage='evaluated'))
    print(f"size perso_app import: {len(perso_app)}")

    ######################################
    print(f"\n### COUNTRY shift iso2 to iso3")
    for el in ['birth_country_code','nationality_country_code','host_country_code','sending_country_code']:
        perso_part = country_iso_shift(perso_part, el, iso2_to3=True)

    ####################################
    print(f"\n### TITLE cleaning")
    def title_clean(df):
        df.loc[~df['title'].isnull(), 'title_clean'] = df.loc[~df['title'].isnull(), 'title'].str.replace(r"[^\w\s]+", " ", regex=True)
        df.loc[~df['title_clean'].isnull(), 'title_clean'] = df.loc[~df['title_clean'].isnull(), 'title_clean'].str.replace(r"\s+", " ", regex=True).str.strip()
        df.mask(df == '', inplace=True)
        return df

    perso_part = title_clean(perso_part)
    perso_app = title_clean(perso_app)

    ###############################
    print(f"\n### STRING cleaning")
    # def prop_string(tab):
    #     from unidecode import unidecode
    #     cols = ['role', 'first_name', 'last_name','title_clean', 'gender']
    #     tab[cols] = tab[cols].map(lambda s:s.casefold() if type(s) == str else s)
                
    #     for i in cols:
    #         tab.loc[~tab[i].isnull(), i] = tab.loc[~tab[i].isnull(), i].str.replace(r"[^\w\s]+", " ", regex=True)
    #         tab.loc[~tab[i].isnull(), i] = tab.loc[~tab[i].isnull(), i].apply(unidecode)
    #     return tab

    cols = ['role', 'first_name', 'last_name','title_clean', 'gender']
    perso_part = prop_string(perso_part, cols)
    perso_app = prop_string(perso_app, cols)

    ##########
    print(f"\n### CONTACT create")
    def contact_name(df):
        for f in ['first_name', 'last_name']:
            df[f] = df[f].fillna('')
            df[f] = df[f].str.strip().str.replace(r"\s+", '-', regex=True)
            df[f] = df[f].str.strip().str.replace(r"-{2,}", '-', regex=True)

        df['contact'] = df.first_name.astype(str).str.lower() + ' ' + df.last_name.astype(str).str.lower()
        
        str_remove=['not applicable']
        df['contact'] = df['contact'].str.strip().str.replace(r"\^s+$", '-', regex=True)
        df = df.loc[~df.contact.isin(str_remove)]
        return df

    perso_app = contact_name(perso_app)
    perso_part = contact_name(perso_part)


    # ###########
    print(f"\n### PIC empty fix")
    # generalPic empty ; replace by pic or fill by generalPic participation
    def empty_pic(df, participation, stage):
        if any(df.generalPic.isnull()):
            print(f"1 - size rows with generelPic null for {stage}: {len(df[df.generalPic.isnull()])}")
            df.loc[df.generalPic.isnull(), 'generalPic'] = df.loc[df.generalPic.isnull(), 'pic']

            # gestion empty generalPic for principal investigator
            x=df.loc[(df.generalPic.isnull())&(df.role=='principal investigator')].project_id.unique()
            if x.size>0:
                y=participation.loc[(participation.project_id.isin(x))&(participation.stage==stage), ['project_id', 'generalPic']]
                df=df.merge(y, how='left', on=['project_id'], suffixes=('', '_y'))
                df.loc[(df.generalPic.isnull())&(~df.generalPic_y.isnull()), 'generalPic'] = df.loc[(df.generalPic.isnull())&(~df.generalPic_y.isnull()), 'generalPic_y'] 
                df.drop(columns='generalPic_y', inplace=True)
                print(f"2 - size rows with generelPic null for {stage}: {len(df[df.generalPic.isnull()])}")
        print(f"size df_{stage} after empty_pic: {len(df)}")
        return df

    perso_part = empty_pic(perso_part, participation, 'successful')
    perso_app = empty_pic(perso_app, participation, 'evaluated')

    ################
    print(f"\n### CALCULATION measures")
    def perso_measure(df):
        df['nb_pic_unique']=df.groupby(['project_id'])['generalPic'].transform('nunique') #combien de pics / projet
        df['nb_name_unique']=df.groupby(['project_id'])['last_name'].transform('nunique') #combien de pics / projet
        df['nb_row_by_pic']=df.groupby(['project_id', 'generalPic'])['last_name'].transform('count') #combien de lignes par pic
        df['nb_name_unique_by_pic']=df.groupby(['project_id', 'generalPic'])['last_name'].transform('nunique')
        df['nb_row_by_pic_name'] = df.groupby(['project_id', 'generalPic','last_name'])['last_name'].transform('count')
        df['nb_row_by_pic_name_unique'] = df.groupby(['project_id', 'generalPic','last_name'])['last_name'].transform('nunique')
        df['nb_pic_by_contact_unique'] = df.groupby(['project_id','contact'])['generalPic'].transform('count')
        
        # print(f"size df: {len(df)}\ncolumns:{df.columns}")
        print(f"size df: {len(df)}")
        return df

    perso_part = perso_measure(perso_part)
    perso_app = perso_measure(perso_app)

    ################
    print(f"\n### without PIC remove")
    def generaPic_remove(df):
        return df.loc[~((df.nb_pic_unique>0)&(df.generalPic.isnull()))]

    perso_part = generaPic_remove(perso_part)
    perso_app = generaPic_remove(perso_app)

    ##############################
    print(f"\n### NAME duplicated remove")
    def name_duplicated_remove(df):
        #### cleaning name duplicated by project 
        ## if by project single name but several rows
        # x[x.project_id=='101039481']

        print(df.role.unique())
        keep_order=['principal investigator', 'fellow', 'main_contact']
        if len(df.role.unique()) > len(keep_order):
            print(f"2 - Attention ! un role nouveau dans perso -> {set(df.role.unique())-set(keep_order)}")

        tmp=pd.DataFrame()
        mask=[(df.nb_row_by_pic_name_unique==1)&(df.nb_row_by_pic_name>1)]
        for i in mask:
            x=df.loc[i]
            print(f"3 - size x before remove: {len(x)}")
            x=x.groupby(['project_id','generalPic', 'last_name']).apply(lambda i: i.sort_values('role', key=lambda col: pd.Categorical(col, categories=keep_order, ordered=True)), include_groups=True).reset_index(drop=True)
            for v in ['title', 'gender','phone','email','birth_country_code','nationality_country_code','host_country_code','sending_country_code']:
                if v in x.columns:
                    x[v]=x.groupby(['project_id', 'generalPic', 'last_name'])[v].bfill()
            x=x.groupby(['project_id', 'generalPic', 'last_name']).head(1)
            print(f"3 - size x after remove: {len(x)}")

            tmp=pd.concat([tmp, x], ignore_index=True)

        df=df.merge(tmp[['project_id', 'generalPic', 'last_name']].drop_duplicates(), how='outer', on=['project_id', 'generalPic', 'last_name'], indicator=True).query('_merge=="left_only"')
        df=pd.concat([df, tmp], ignore_index=True)

        if len(df)==0:
            print(f"ATTENTION table vide après traitement name_duplicated_remove")
        else:
            print(f"size après traitement name_duplicated_remove: {len(df)}")

        return df.drop(columns=['_merge'])

    perso_part = name_duplicated_remove(perso_part)
    perso_app = name_duplicated_remove(perso_app)
    # ####################################
    perso_part = perso_measure(perso_part)
    perso_app = perso_measure(perso_app)

    print(f"\n### PI duplicated")
    def PI_duplicated(df):
        if any(df.role=='principal investigator'):
            # select if same person and one PI in a single project 
            mask=(df.nb_pic_by_contact_unique>1)&(df.role=='principal investigator')
            pi=df.loc[mask, ['project_id', 'contact']].drop_duplicates().merge(df, how='inner')
            pi['role'] = 'principal investigator'
            for v in ['title', 'gender','birth_country_code','nationality_country_code','sending_country_code']:
                if v in df.columns:
                    pi=pi.sort_values(v)
                    pi[v]=pi.groupby(['project_id', 'contact'])[v].ffill()
            
            df=df.merge(pi[['project_id', 'generalPic', 'contact']].drop_duplicates(), how='outer', on=['project_id', 'generalPic', 'contact'], indicator=True).query('_merge=="left_only"')
            df=pd.concat([df, pi], ignore_index=True)
            print(f"-size df after cleaning pi_duplicated: {len(df)}")
            return df.drop(columns=['_merge'])
        
    perso_part=PI_duplicated(perso_part)

    #######################
    print(f"\n### PARTICIPATION+PERSO")
    def perso_participation(df, participation, project, entities, stage):
        
        df=df.loc[df.project_id.isin(participation[participation.stage==stage].project_id.unique())]
        df=df.merge(participation.loc[participation.stage==stage, ['project_id', 'generalPic', 'country_code']], how='outer', on=['project_id', 'generalPic'], indicator=True).query('_merge!="right_only"')
        df.loc[df._merge=='left_only', 'shift'] = 'past'

        if stage=='successful':
            df.loc[(df._merge=='both')&(df.host_country_code.isnull()), 'host_country_code'] = df.loc[(df._merge=='both')&(df.host_country_code.isnull()), 'country_code']

        df=df.merge(project.loc[project.stage==stage, ['project_id', 'call_year', 'thema_code', 'destination_code']], how='inner', on=['project_id'])

        x=entities[['entities_id', 'entities_name', 'generalPic', 'id_secondaire', 'country_code', 'country_code_mapping']].drop_duplicates()
        df=df.merge(x, how='left', on=['generalPic', 'country_code'])

        if len(df)==0:
            print(f"ATTENTION table vide après lien avec participation")
        else:
            print(f"size app lien avec participation clean : {len(df)}\ncolumns:{df.columns}")
        return df.drop(columns=['_merge'])

    perso_part = perso_participation(perso_part, participation, project, entities, 'successful')
    perso_app = perso_participation(perso_app, participation, project, entities, 'evaluated')

    def iso2_add(df):
        df = (df.merge(my_countries[['iso2', 'iso3']].drop_duplicates(), how='left', left_on='country_code', right_on='iso3')
                .drop(columns='iso3')
        )
        if any(df.iso2.isnull()):
            print(f"country iso2 missing for iso3 -> {df[df.iso2.isnull()].country_code.unique()}")
        return df
    perso_part = iso2_add(perso_part)
    perso_app = iso2_add(perso_app)

    # ##################
    print(f"\n### PHONE cleaning")
    def phone_clean(df):
        y = df.loc[(df.country_code=='FRA')&(~df.phone.isnull()), ['phone']]
        y['tel_clean']=y.phone.str.replace(r"(^\++[0-9]{1,3}\s+)", '', regex=True)
        y['tel_clean']=y.tel_clean.str.replace(r"[^0-9]+", '', regex=True)
        y['tel_clean']=y.tel_clean.str.replace(r"^(33|033|0033)", '', regex=True).str.rjust(10, '0')
        y.loc[(y.tel_clean.str.len()>10)&(y.tel_clean.str[0:1]=='0'), 'tel_clean'] = y.tel_clean.str[0:10]
        y['tel_clean']=y.tel_clean.str.replace(r"^0+$", '', regex=True)
        # work_csv(y, 'tel_perso')
        return pd.concat([df, y[['tel_clean']]], axis=1)

    perso_part = phone_clean(perso_part)
    perso_app = phone_clean(perso_app)

    # #######################
    print(f"\n### MAIL cleaning")
    def mail_clean(df):
        mail_del=["gmail", "yahoo", "hotmail", "wanadoo", "aol", "free", "skynet", "outlook", "icloud", "googlemail"]

        df['domaine'] = df.email.str.split('@').str[1].str.split('.').str[:-1].fillna('').apply(' '.join)
        tmp = df.loc[~df.domaine.isnull(), ['domaine']]

        for el in mail_del:
            m = r"^"+el+r"($|\s)"
            tmp.loc[tmp['domaine'].str.contains(m, case=True, flags=0, na=None, regex=True) == True, 'domaine_email'] = ''
            tmp.loc[tmp['domaine_email'].isnull(), 'domaine_email'] = tmp['domaine']

        return pd.concat([df, tmp], axis=1).drop(columns='domaine')

    perso_app = mail_clean(perso_app)
    perso_part = mail_clean(perso_part)
    ##############

    def nationality_clean(df):
        filter_df=df.loc[(df.nationality_country_code.isnull()), ['generalPic', 'contact']].drop_duplicates().assign(fill_nat=True)
        df=df.merge(filter_df, how='left', on=['generalPic', 'contact'])
        df['rows_by_picContact']=df.groupby(['generalPic', 'contact'], dropna=False)['nationality_country_code'].transform('nunique')
        df.loc[(df.fill_nat==True)&(df.rows_by_picContact==1), 'nationality_country_code']=df.loc[(df.fill_nat==True)&(df.rows_by_picContact==1)].sort_values(['generalPic', 'contact', 'nationality_country_code']).groupby(['generalPic', 'contact'], group_keys=True)['nationality_country_code'].ffill()
        df.drop(columns='rows_by_picContact', inplace=True)
        return df
    
    perso_part = nationality_clean(perso_part)
    #################

    # def orcid_id_fill(df):
    #     print("### orcid fillna")
    #     temp=df.groupby(['generalPic', 'contact', 'domaine_email'], dropna=False)['orcid_id'].nunique(dropna=False).reset_index()
    #     print(temp[temp.orcid_id>2])
    #     temp=temp[temp.orcid_id>1].drop(columns='orcid_id')
    #     df=df.merge(temp, how='left', on=['generalPic', 'contact', 'domaine_email'], indicator=True)
    #     df.loc[df._merge=='both', 'orcid_id'] = df.loc[df._merge=='both'].sort_values(['generalPic', 'contact', 'domaine_email', 'orcid_id']).groupby(['generalPic', 'contact', 'domaine_email'], group_keys=True)['orcid_id'].ffill()
    #     return df.drop(columns='_merge')
    
    # perso_app = orcid_id_fill(perso_app)
    # #################

    # # add orcid_id (perso_app) into perso_part
    # print(f"\n### INFO missing between datasets")
    # perso_part=perso_part.merge(perso_app[['project_id', 'contact', 'orcid_id']], how='left', on=['project_id', 'contact']) 
    # perso_app=perso_app.merge(perso_part[['project_id', 'contact', 'nationality_country_code']], how='left', on=['project_id', 'contact'])
    # ##################
    
    # # fill missing value with other df part/app
    # print(f"\n### GENDER/TITLE missing")
    # def gender_title_missing(part, app):
    #     tab=(part[['project_id', 'contact', 'gender','title_clean']]
    #     .merge(app[['project_id', 'contact', 'gender', 'title_clean']], 
    #             how='inner', on=['project_id', 'contact'], suffixes=('_x','_y'))
    #             .drop_duplicates())
    #     cl=['gender', 'title_clean']
    #     for i in cl:
    #         if any(tab.loc[(tab[f"{i}_x"].isnull())&(~tab[f"{i}_y"].isnull())]):
    #             tab.loc[(tab[f"{i}_x"].isnull())&(~tab[f"{i}_y"].isnull()), f"{i}_x"] = tab[f"{i}_y"]
    #         if any(tab.loc[(~tab[f"{i}_x"].isnull())&(tab[f"{i}_y"].isnull())]):
    #             tab.loc[(~tab[f"{i}_x"].isnull())&(tab[f"{i}_y"].isnull()), f"{i}_y"] = tab[f"{i}_x"]

    #         part = part.merge(tab[['project_id', 'contact', f"{i}_x"]].drop_duplicates(), how='left', on=['project_id', 'contact'])
    #         part.loc[part[i].isnull(), i] = part.loc[part[i].isnull(), f"{i}_x"]
    #         part.drop(columns=f"{i}_x", inplace=True)
    #         app = app.merge(tab[['project_id', 'contact', f"{i}_y"]].drop_duplicates(), how='left', on=['project_id', 'contact'])
    #         app.loc[app[i].isnull(), i] = app.loc[app[i].isnull(), f"{i}_y"]
    #         app.drop(columns=f"{i}_y", inplace=True)

    #     return part, app

    # perso_part, perso_app = gender_title_missing(perso_part, perso_app)

    # print(f"\n### EXPORT final datasets")
    # (perso_part[['project_id', 'generalPic', 'role', 'first_name', 'last_name',
    #     'title_clean', 'gender', 'email', 'tel_clean', 'domaine_email', 'orcid_id', 'birth_country_code',
    #     'nationality_country_code', 'host_country_code', 'sending_country_code', 'iso2',
    #     'stage', 'contact', 'country_code', 'shift', 'call_year', 'thema_code', 'destination_code',
    #     'entities_id', 'entities_name','id_secondaire', 'country_code_mapping']]
    #     .drop_duplicates()
    #     .to_pickle(f"{PATH_CLEAN}persons_participants.pkl"))

    # (perso_app[['project_id', 'generalPic', 'role', 'first_name', 'last_name', 'nationality_country_code',
    #     'title_clean', 'gender', 'tel_clean', 'email', 'domaine_email', 'researcher_id', 'orcid_id',
    #     'google_scholar_id', 'scopus_author_id', 'stage', 'iso2',
    #     'contact', 'country_code', 'shift', 'call_year', 'thema_code', 'destination_code',
    #     'entities_id', 'entities_name','id_secondaire', 'country_code_mapping']]
    #     .drop_duplicates()
    #     .to_pickle(f"{PATH_CLEAN}persons_applicants.pkl"))
    return perso_app
# df=persons_preparation(CSV_DATE)

In [ ]:
PATH_PERSONS=f"{PATH_API}persons/"
# result=pd.read_pickle(f"{PATH_PERSONS}persons_{CSV_DATE}.pkl")

In [ ]:
result[result.display_name=='anwesha sarkar']

In [ ]:
temp=df.groupby(['generalPic', 'contact'], dropna=False)['orcid_id'].nunique(dropna=False).reset_index()
print(temp[temp.orcid_id>2])
# temp=temp[temp.orcid_id>1].drop(columns='orcid_id')

In [ ]:
df[(df.contact.str.contains('feng'))&(df.entities_name.str.contains('Leeds'))][['project_id', 'generalPic','role', 'orcid_id', 'stage', 'contact', 'country_code',
       'shift', 'call_year', 'thema_code', 'destination_code', 'entities_id',
       'entities_name', 'id_secondaire', 'country_code_mapping', 'iso2',
       'tel_clean', 'domaine_email']]

In [ ]:
lvar=['project_id', 'generalPic', 'role', 
       'title_clean', 'gender', 'email', 'tel_clean', 'domaine_email',
       'orcid_id', 'birth_country_code', 'nationality_country_code',
       'host_country_code', 'sending_country_code', 'iso2', 'stage', 'contact',
       'country_code', 'shift', 'call_year', 'thema_code', 'destination_code',
       'entities_id', 'entities_name', 'id_secondaire', 'country_code_mapping']
pp = pd.concat([perso_part.drop_duplicates(), perso_app.drop_duplicates()], ignore_index=True)

mask=((pp.country_code=='FRA')|(pp.nationality_country_code=='FRA')|(pp.destination_code.isin(['COG', 'PF', 'STG', 'ADG', 'POC','SyG', 'PERA', 'SJI'])))&(~(pp.contact.isnull()&pp.orcid_id.isnull()))
pp=pp.loc[mask, lvar].sort_values(['country_code','orcid_id'], ascending=False).drop_duplicates()
pp['contact2']=pp.contact.str.replace('-', ' ')

In [ ]:
def persons_files_import(thema, PATH_PERSONS):
    import re, os
    fname=''.join([filename for filename in os.listdir(PATH_PERSONS) if thema in filename])
    print(fname)

    if fname:
        with open(f"{PATH_PERSONS}{fname}", 'rb') as f:
            return pickle.load(f)

    if fname == []:
        fmax=max(int(os.path.splitext(filename)[0].split('_')[-1]) for filename in os.listdir(PATH_PERSONS) if re.search(r"persons_authors_[0-9]+",filename))
        if fmax:
            with open(f"{PATH_PERSONS}persons_authors_{fmax}.pkl", 'rb') as f:
                return pickle.load(f)

In [ ]:
with open(f'{PATH_PERSONS}persons_authors_erc_{CSV_DATE}.pkl', 'rb') as f:
    pers_ers = pickle.load(f)

In [ ]:
with open(f'{PATH_PERSONS}persons_authors_{CSV_DATE}.pkl', 'rb') as f:
    pers_api = pickle.load(f)

In [ ]:
def persons_api_simplify(df):
    pers = [] 
    for p in df:
        # elem = {k: v for k, v in p.items() if (v and v != "NaT")}

        p['institutions'] = []
        if p.get("affiliations"):
            for aff in p["affiliations"]:  
                res={"institution_name":aff.get('institution').get("display_name"),
                "institution_ror":aff.get('institution').get("ror"),
                "institution_country2":aff.get('institution').get("country_code"),
                "years":aff.get("years")}
                p['institutions'].append(res)
    
        p["orcid_openalex"] = p["ids"].get("orcid")            

        delete=['display_name_alternatives', 'topics', 'affiliations', 'id', 'last_known_institutions', 'ids']
        for field in delete:
            if p.get(field):
                p.pop(field)

        # elem = {k: v for k, v in elem.items() if (v and v != "NaT")}
        pers.append(p)

    print(len(pers))
    return pers

pers = persons_api_simplify(pers_api)
perc = persons_api_simplify(pers_ers)

In [ ]:
perc

In [ ]:
def persons_results_clean(df):
    from unidecode import unidecode
    from functions_shared import my_country_code, prop_string

    # df=pd.json_normalize(df, max_level=1)
    df=pd.json_normalize(df, record_path=['institutions'], meta=['match', 'orcid', 'display_name', 'orcid_openalex'], errors='ignore')
    df=df[~df.astype(str).duplicated()]
    cols = ['display_name']
    df = prop_string(df, cols)

    df['rows_by_name_orcid'] = df.groupby(['display_name', 'orcid'], dropna=False).transform('size')

    # persName_withOrcid_noAff=df[(df.match=='full_name')&(~df.orcid.isnull())&(df.institution_name.isnull())]
    # print(f"size person detect by name with an orcid but no affiliations: {len(persName_withOrcid_noAff)}")


    for i in ['orcid_openalex', 'orcid', 'institution_ror']:
        df.loc[~df[i].isnull(), i] = df.loc[~df[i].isnull()][i].str.split("/").str[-1]
    df['institution_ror'] = 'R'+ df['institution_ror'].astype(str)

    df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
    df=df[['match', 'display_name', 'orcid_openalex', 'years', 'institution_ror', 'institution_name', 'institution_country2', 'rows_by_name_orcid']]
    my_countries=my_country_code()
    df=(df.merge(my_countries[['iso2', 'iso3', 'parent_iso3']].drop_duplicates(), 
                 how='left', left_on='institution_country2', right_on='iso2')
        .drop(columns=['iso2'])
        .rename(columns={'iso3':'institution_country_map', 'parent_iso3':'institution_country'})
        )

    from step8_referentiels.paysage import paysage_prep
    from config_path import PATH
    DUMP_PATH=f'{PATH}referentiel/'
    paysage = paysage_prep(DUMP_PATH)
    df=(df.merge(paysage[['nom_long', 'numero_ror', 'numero_paysage', 'country_code_map', 'num_nat_struct']].drop_duplicates(), 
                 how='left', left_on='institution_ror', right_on='numero_ror'))



#     print(f"-3 size {match} cleaned: {len(df)}")
    return df

perc=persons_results_clean(perc)
pers=persons_results_clean(pers)


In [ ]:
temp=pd.concat([perc, pers], ignore_index=True)
temp=temp[~temp.astype(str).duplicated()]
pd.to_pickle(temp,f"{PATH_PERSONS}persons_{CSV_DATE}.pkl")
len(temp)

In [ ]:
pers.columns

In [ ]:
# temp=temp[~temp.astype(str).duplicated()]
pd.to_pickle(temp,f"{PATH_PERSONS}persons_{CSV_DATE}.pkl")
len(temp)

In [ ]:
# oth=pp.loc[~pp.thema_code.isin(['ERC', 'MSCA'])].drop_duplicates().reset_index(drop=True).sort_values('contact2')
# # pers_oth[pers_oth.display_name.str.contains('aubert')].display_name.unique()
# df=oth.merge(temp, how='inner', left_on=['contact2', 'country_code'], right_on=['display_name','iso3'])
# df=df[~df.astype(str).duplicated()]
# # df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
# df['filt']=df.apply(lambda x: x['call_year'] in x['years'], axis=1)

In [ ]:
# pers_oth[pers_oth.display_name.str.contains('aubert')].display_name.unique()
df=oth.merge(temp, how='inner', left_on=['contact2', 'country_code'], right_on=['display_name','iso3'])
df=df[~df.astype(str).duplicated()]
# df['years']=df['years'].map(lambda liste: ';'.join(str(x) for x in liste))
df['filt']=df.apply(lambda x: x['call_year'] in x['years'], axis=1)

df=df[['project_id', 'generalPic', 'role', 'orcid_id', 'orcid_openalex',
       'nationality_country_code', 'contact2',
        'stage', 'contact', 'display_name','country_code', 
        'entities_id', 'entities_name',
       'id_secondaire', 'country_code_mapping', 
        'institution_ror', 'institution_name',
       'institution_country',  'iso3', 'parent_iso3',
       'nom_long', 'numero_ror', 'numero_paysage', 'country_code_map',
       'num_nat_struct','rows_by_name_orcid','call_year','years', 'filt','thema_code', 'destination_code']]
# 

#extract entities_id begun by PIC
work_csv(df.loc[df.entities_id.str.startswith('pic', na=False), 
                ['generalPic', 'entities_name','id_secondaire', 'institution_ror', 'institution_name',
                'institution_country','numero_ror', 'numero_paysage', 'num_nat_struct']].drop_duplicates(), 'verif_id_struct_from openalex')

# #keep only rows with th same year
df1=df.loc[(df.filt==True)]


# df=df.loc[(df.filt==True)&(~df.num_nat_struct.isnull())]



In [ ]:
work_csv(df.loc[(df.filt==True)&(~df.num_nat_struct.isnull())], 'pers_with_nns')


In [ ]:
pp = pd.concat([perso_part.drop_duplicates(), perso_app.drop_duplicates()], ignore_index=True)
# mask=((pp.country_code=='FRA')|(pp.nationality_country_code=='FRA')|(pp.destination_code.isin(['COG', 'PF', 'STG', 'ADG', 'POC','SyG', 'PERA', 'SJI'])))&(~(pp.contact.isnull()&pp.orcid_id.isnull()))
pp=pp.sort_values(['country_code','orcid_id'], ascending=False).drop_duplicates()
pp['contact2']=pp.contact.str.replace('-', ' ')
len(pp)

In [ ]:
pp.loc[pp.contact=='zoubir khatir', ['stage','project_id', 'generalPic', 'role', 
       'title_clean', 'gender', 'email', 'tel_clean',
       'orcid_id', 'nationality_country_code',
       'host_country_code', 'contact',
       'country_code', 'call_year', 'thema_code', 'destination_code',
       'entities_id', 'entities_name', 'id_secondaire', 'country_code_mapping'
       ]]

In [ ]:
temp.loc[temp.display_name=='zoubir khatir']

In [ ]:
df=pp.merge(temp, how='inner', left_on=['contact2', 'country_code'], right_on=['display_name','iso3'])
df['filt_year']=df.apply(lambda x: x['call_year'] in x['years'], axis=1)
# same year
df=df.loc[(df.filt_year==True)]
#same entities
df1=df.loc[(df.entities_id==df.numero_paysage)]
df1=df1.assign(orcid_clean=np.where(df1.orcid_openalex.isnull(), df1.orcid_id, df1.orcid_openalex)).drop(columns='orcid_openalex').drop_duplicates()
# df1

In [ ]:
work_csv(df1, 'verif_orcid')

In [ ]:
# import time, pickle, requests
# from step7_persons.affiliations import openalex_name, openalex_orcid
# from config_path import PATH_CLEAN, PATH_API, PATH_WORK
# print(time.strftime("%H:%M:%S"))
# rlist=[]

# try:


#     author = {
#     "name": "caye pierre'",
#     "orcid": ""
#     }

#     if author.get("orcid"):
#         result = openalex_orcid(author)
#         if result.get('match'):
#             rlist.append(result)
#         else:
#             result = openalex_name(author)
#             if result:
#                 rlist.extend(result)
#     if author.get("orcid")=='':
#         result = openalex_name(author)
#         if result:
#             rlist.extend(result)

#     nf=f"persons_author"
#     with open(f'{PATH_WORK}test.pkl', 'wb') as f:
#         pickle.dump(rlist, f)
#     print(time.strftime("%H:%M:%S"))

# except requests.exceptions.HTTPError as http_err:
#     print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
#     nf=f"persons_author"
#     with open(f'{PATH_WORK}test.pkl', 'wb') as f:
#         pickle.dump(rlist, f)
#     print(time.strftime("%H:%M:%S"))

In [ ]:

with open(f'{PATH_WORK}test.pkl', 'rb') as f:
    test = pickle.load(f)
test

## return results from apenalex

1 - à partir d'orcid
2 - à partir du nom car orcid pas dans openalex -> vérifier que même personne ?
3 - à partir du nom car orcid non renseigné


In [ ]:
pers_api.mask(pers_api=='', inplace=True)
tmp1=pers_api.merge(my_countries, how='left', left_on='country_code', right_on='iso2')
# pers_api=country_clean(pers_api, ['country_code'])
tmp1=pers_api.loc[pers_api.match=='orcid']
len(tmp1)


In [ ]:
print(f"{len(perso_part)} ; {perso_part.columns}")

In [ ]:
tmp_part=perso_part.merge(tmp1, how='inner', left_on=['orcid_id'], right_on=['orcid'], suffixes=('','_api'))
tmp_part.loc[tmp_part.country_code!=tmp_part.country_code_api]

In [ ]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
data

In [ ]:
def openalex_name(author):
    url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
    nb_openalex=requests.get(url).json().get("meta").get('count')
    result={}
    if nb_openalex>0:
        for n in range(nb_openalex): 
            author_openalex = requests.get(url).json().get("results")[n]
            result.update({'display_name':author_openalex.get('display_name'), 'openalex_id':author_openalex.get('id'), 'affiliations':author_openalex.get('affiliations'), 'topics':author_openalex.get('topics'),  'x_concepts':author_openalex.get('x_concepts'), 'ids':author_openalex.get('ids'), 'display_name_alternatives':author_openalex.get('display_name_alternatives')})
            author.update(result)
            df=pd.concat([df, pd.json_normalize(author)])
    return df

In [ ]:
from config_path import PATH_API
import os, pandas as pd
 

for racine, repertoires, fichiers in os.walk(PATH_API):
    print(f"{racine}, {repertoires}, {fichiers}")
    for fichier in fichiers:
        if fichier.startswith('persons'):
            print(os.path.join(racine, fichier))
            globals()[f"{fichier}"]= pd.read_pickle(os.path.join(racine, fichier))


In [ ]:
fl=['persons_author.pkl', 'persons_author_10000.pkl', 'persons_author_12000.pkl', 'persons_author_2000.pkl', 'persons_author_4000.pkl', 'persons_author_6000.pkl', 'persons_author_8000.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']
for racine, repertoires, fichiers in os.walk(PATH_API):
    for i in fl:
        name=f"{i}".split('.')[0]
        print(name)
        globals()[name] = pd.read_pickle(os.path.join(racine, fichier))

In [ ]:
pers=pd.concat([persons_author_10000, persons_author_12000, persons_author_2000, persons_author_4000, persons_author_6000, persons_author_8000], ignore_index=True)

fl=['persons_author.pkl', 'persons_author_name.pkl', 'persons_author_orcid.pkl']

In [ ]:
def openalex_orcid(author):
    # from config_api import openalex_usermail
    import requests, time
    try:
        url = f"https://api.openalex.org/authors/orcid:{author.get('orcid')}?mailto=zmenesr@gmail.com"
        author_openalex = requests.get(url).json()
        result = author | {'display_name':author_openalex.get('display_name'), 
                           'openalex_id':author_openalex.get('id'), 
                           'affiliations':author_openalex.get('affiliations'), 
                           'topics':author_openalex.get('topics'),  
                           'x_concepts':author_openalex.get('x_concepts'), 
                           'ids':author_openalex.get('ids'), 
                           'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                           'match':'orcid'}
        return result
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author

In [ ]:
def openalex_name(author):
    import time
    try:
        url = f"https://api.openalex.org/authors?filter=display_name.search:{author.get('name')}"
        nb_openalex=requests.get(url).json().get("meta").get('count')
        dl=[]
        if nb_openalex>0:
            print(nb_openalex)
            for n in range(nb_openalex): 
                author_openalex = requests.get(url).json().get("results")[n]
                if author_openalex.get('affiliations')!=[]:
                    result=author | {'display_name':author_openalex.get('display_name'), 
                                    'openalex_id':author_openalex.get('id'), 
                                    'affiliations':author_openalex.get('affiliations'),
                                    'topics':author_openalex.get('topics'),
                                    'x_concepts':author_openalex.get('x_concepts'), 
                                    'ids':author_openalex.get('ids'), 
                                    'display_name_alternatives':author_openalex.get('display_name_alternatives'),
                                    'match':'name'}
                    dl.append(result)
        return dl
    
    except requests.exceptions.HTTPError as http_err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> HTTP error occurred: {http_err}")
        return author
    except requests.exceptions.RequestException as err:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> Error occurred: {err}")
        return author           
    except Exception as e:
        print(f"\n{time.strftime("%H:%M:%S")}, {author}-> An unexpected error occurred: {e}")
        return author
    

In [ ]:
from config_path import PATH_CLEAN
# from functions_shared import chunkify
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
# pp = pd.concat([perso_part[['contact', 'orcid_id', 'country_code']].drop_duplicates(), perso_app[['contact', 'orcid_id', 'country_code']].drop_duplicates()], ignore_index=True)
pp = perso_part[['contact', 'orcid_id', 'country_code']].fillna('')
pp=pp.loc[(pp.country_code=='FRA')].sort_values('orcid_id', ascending=False)
pp
data_chunks=list(chunkify(pp, 10000))
for i in range(0, len(data_chunks)):
    print(f"Loop {i}, size data_chunks: {len(data_chunks)}")
    # print(type(data_chunks))
    df_temp = data_chunks[i]
    

In [ ]:
import pickle, pandas as pd
from config_path import PATH_API
pers_api=[]
for i in range(1,3):
    print(i)
    with open(f"{PATH_API}fr_persons_author_{i}.pkl", 'rb') as f:
        globals()[f"pers_api{i}"] = pickle.load(f)
    pers_api.extend(globals()[f"pers_api{i}"])


In [ ]:
pers_api=pd.json_normalize(pers_api1, record_path=['affiliations'], meta=['name', 'orcid', 'display_name', 'openalex_id',  'match',  ["ids", "orcid"]],
        errors='ignore')

pers_api.columns = pers_api.columns.str.replace(r"[.*_]+", '_', regex=True)

pers_api = (pers_api
            .rename(columns={
                    'institution_country_code':'country_code'})
            .drop(columns=['institution_type','institution_lineage']))

for i in ['ids_orcid', 'institution_ror']:
    pers_api.loc[~pers_api[i].isnull(), i] = pers_api.loc[~pers_api[i].isnull()][i].str.split("/").str[-1]

In [ ]:
from config_path import PATH_CLEAN
perso_part = pd.read_pickle(f"{PATH_CLEAN}persons_participants.pkl")
perso_app = pd.read_pickle(f"{PATH_CLEAN}persons_applicants.pkl")

In [ ]:
with open(f"{PATH_API}persons_author.pkl", 'rb') as f:
    author_orcid = pickle.load(f)

In [ ]:
author_orcid=pd.json_normalize(author_orcid, record_path=['affiliations'], meta=['name','orcid', 'display_name', 'ids', 'match'])
author_orcid